In [1]:
%load_ext autoreload
%autoreload 2

## English Rest Semeval-2016 SB1 Slot3

In [11]:
%%writefile config.json
{
    "data_config": {
        "competition": "semeval",
        "domain": "rest",
        "language": "en",
        "test_filename": "/Volumes/My Passport/Datasets/Sentiment/ABSA16/ABSA16_Restaurants_En_Test.xml",
        "train_filename": "/Volumes/My Passport/Datasets/Sentiment/ABSA16/ABSA16_Restaurants_En_Train.xml"
    },
    "model_config": {
        "is_sequence_predictor": true,
        "char_dropout_p": 0.4,
        "char_embedding_dim": 4,
        "char_function_output_size": 30,
        "char_max_word_length": 30,
        "dense_size": 32,
        "dense_dropout": 0.4,
        "gram_dropout_p": 0.4,
        "gram_hidden_size": 32,
        "rnn_bidirectional": true,
        "rnn_dropout_p": 0.5,
        "rnn_hidden_size": 32,
        "rnn_output_dropout_p": 0.4,
        "rnn_n_layers": 2,
        "use_chars":  false,
        "use_crf": true,
        "use_pos": true,
        "use_word_embeddings": true,
        "word_embedding_dim": 300,
        "word_embedding_dropout_p": 0.4,
        "output_size": 25
    },
    "batch_size": 8,
    "embeddings_filename": "semeval-2016-en-rest-fasttext.txt",
    "epochs": 100,
    "lr": 0.001,
    "max_length": 650,
    "output_filename": "submission.xml",
    "patience": 2,
    "task_type": "12",
    "use_pretrained_embedding": true,
    "val_size": 0.2,
    "word_max_length": 30,
    "model_filename": "model.pt"
}

Overwriting config.json


In [4]:
from src.config import Config
from src.parser import get_data

config = Config()
config.load("config.json")
train_data, test_data = get_data(config.data_config)   

max_length = max(train_data.get_lengths() + test_data.get_lengths())
vocabulary = train_data.get_vocabulary().merge(test_data.get_vocabulary())
char_set = train_data.get_char_set()
print(vocabulary.size())
print(char_set)

4091
 ,A:8'$M5o9BmO2/6r‘tf%SKuFPTNQX*3)cj4@p!(sRia;nb.G=+yYe"L?ZxH0–dD#g-kzJWCEUé’7lw1IvhVq


In [12]:
from src.train import train_model
from src.config import get_targets_additionals

targets, additionals = get_targets_additionals(train_data)
model = train_model("config.json", train_data, vocabulary, char_set, targets, additionals)

{'AMBIENCE#GENERAL': 0, 'DRINKS#PRICES': 1, 'DRINKS#QUALITY': 2, 'DRINKS#STYLE_OPTIONS': 3, 'FOOD#PRICES': 4, 'FOOD#QUALITY': 5, 'FOOD#STYLE_OPTIONS': 6, 'LOCATION#GENERAL': 7, 'RESTAURANT#GENERAL': 8, 'RESTAURANT#MISCELLANEOUS': 9, 'RESTAURANT#PRICES': 10, 'SERVICE#GENERAL': 11}
{0: 'AMBIENCE#GENERAL', 1: 'DRINKS#PRICES', 2: 'DRINKS#QUALITY', 3: 'DRINKS#STYLE_OPTIONS', 4: 'FOOD#PRICES', 5: 'FOOD#QUALITY', 6: 'FOOD#STYLE_OPTIONS', 7: 'LOCATION#GENERAL', 8: 'RESTAURANT#GENERAL', 9: 'RESTAURANT#MISCELLANEOUS', 10: 'RESTAURANT#PRICES', 11: 'SERVICE#GENERAL'}
Use cuda:  False


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchcrf/__init__.py:58: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.start_transitions, -0.1, 0.1)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchcrf/__init__.py:59: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.end_transitions, -0.1, 0.1)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchcrf/__init__.py:60: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.transitions, -0.1, 0.1)


Unknown words in semeval-2016-en-rest-fasttext.txt: 218
Epoch: 0, train loss: 4201.781944056919, val loss: 3568.6249321831597
Epoch: 1, train loss: 2959.098163713728, val loss: 3188.3396674262153
Epoch: 2, train loss: 2820.4223039899553, val loss: 3104.1981065538193


KeyboardInterrupt: 

In [ ]:
import torch
import numpy as np

from src.config import Config
from src.model import load_model
from src.train import get_batches
from src.semeval_parser import Opinion, Review, Sentence

def predict(model_filename, config_filename, test_data, vocabulary, targets, additionals):
    config = Config()
    config.load(config_filename)
    
    use_cuda = torch.cuda.is_available()
    model, _ = load_model(model_filename, config_filename, use_cuda)
    model.eval()
    gram_vector_size =len(test_data.reviews[0].sentences[0][0].vector)
    
    task_key = config.data_config.competition + "-" + config.task_type
    test_batches = get_batches(test_data.reviews, vocabulary, char_set, 1, 
                               config.max_length, config.word_max_length, 
                               targets[task_key], additionals[task_key])
    new_reviews = []
    for review, batch in zip(test_data.reviews, test_batches):
        new_review = Review(rid=review.rid)
        for sentence in review.parsed_sentences:
            new_review.parsed_sentences.append(Sentence(sentence.sid, sentence.text))
        predictions = model.predict(batch)
        
        length = sum([int(elem != 0) for elem in batch.word_indices[0].data])
        if model.config.use_crf:
            review_pred = predictions[0][:length]
        else:
            review_pred = predictions[0, :length]

        tokens = [word for sentence in review.sentences for word in sentence]
        aspect = Opinion()
        sid = ''
        done_opinions = set()
        for i, token in enumerate(tokens):
            sid = token.sid
            pred_class = review_pred[i].cpu().item()
            if config.task_type == '12':
                if pred_class % 2 == 0 and pred_class != 0 and aspect.is_empty():
                    pred_class -= 1
                if pred_class % 2 == 1:
                    aspect.words.append(token)
                    category = rev_categories[(pred_class-1)//2]
                    aspect.cat_first, aspect.cat_second = category.split("#")
                    aspect.inflate_target()
                if pred_class % 2 == 0 and pred_class != 0:
                    aspect.words.append(token)
                    category = rev_categories[(pred_class-2)//2]
                    aspect.cat_first, aspect.cat_second = category.split("#")
                    aspect.inflate_target()
                if pred_class == 0 and not aspect.is_empty():
                    aspect.begin = aspect.words[0].begin
                    aspect.end = aspect.words[-1].end
                    aspect.inflate_target()
                    for sentence in new_review.parsed_sentences:
                        if sentence.sid == token.sid:
                            sentence.aspects.append(aspect)
                    new_review.aspects.append(aspect)
                    aspect = Opinion()
            elif config.task_type == '3':
                if not token.opinions:
                    continue
                for opinion in token.opinions:
                    if opinion not in done_opinions:
                        aspect = Opinion(begin=opinion.begin, end=opinion.end, 
                                         polarity=pred_class-1, cat_first=opinion.cat_first,
                                         cat_second=opinion.cat_second,
                                         target=opinion.target.replace('"', "'").replace('&', '#'))
                        done_opinions.add(opinion)
                        for sentence in new_review.parsed_sentences:
                            if sentence.sid == token.sid:
                                sentence.aspects.append(aspect)
                        new_review.aspects.append(aspect)
        if config.task_type == '12' and not aspect.is_empty():
            for sentence in new_review.parsed_sentences:
                if sentence.sid == token.sid:
                    sentence.aspects.append(aspect)
            new_review.aspects.append(aspect)
        new_reviews.append(new_review)
    
    xml = '<?xml version="1.0" ?>\n'
    xml += '<Reviews>\n'
    for review in new_reviews:
        xml += review.to_xml()
    xml += '</Reviews>\n'
    with open(config.output_filename, "w", encoding='utf-8') as f:
        f.write(xml)

predict("model.pt", "config.json", test_data, vocabulary, targets, additionals)
!head -n 1000 submission.xml

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchcrf/__init__.py:58: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.start_transitions, -0.1, 0.1)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchcrf/__init__.py:59: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.end_transitions, -0.1, 0.1)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchcrf/__init__.py:60: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.transitions, -0.1, 0.1)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchcrf/__init__.py:284: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  best_tags = [best_last_tag[0]]


In [14]:
from src.embeddings import shrink_w2v
ALL_EMBEDDINGS_FILENAME = "/Volumes/My Passport/Models/Vectors/FastText/wiki.en.vec"
EMBEDDINGS_FILENAME = "semeval-2016-en-rest-fasttext.txt"
shrink_w2v(ALL_EMBEDDINGS_FILENAME, vocabulary, 10000, EMBEDDINGS_FILENAME)

Parsed words: 0, intersection: 0, unknown words:4091
Parsed words: 100000, intersection: 3487, unknown words:604
Parsed words: 200000, intersection: 3645, unknown words:446
Parsed words: 300000, intersection: 3706, unknown words:385
Parsed words: 400000, intersection: 3738, unknown words:353
Parsed words: 500000, intersection: 3760, unknown words:331
Parsed words: 600000, intersection: 3775, unknown words:316
Parsed words: 700000, intersection: 3782, unknown words:309
Parsed words: 800000, intersection: 3799, unknown words:292
Parsed words: 900000, intersection: 3809, unknown words:282
Parsed words: 1000000, intersection: 3818, unknown words:273
Parsed words: 1100000, intersection: 3826, unknown words:265
Parsed words: 1200000, intersection: 3832, unknown words:259
Parsed words: 1300000, intersection: 3835, unknown words:256
Parsed words: 1400000, intersection: 3840, unknown words:251
Parsed words: 1500000, intersection: 3844, unknown words:247
Parsed words: 1600000, intersection: 3851